### Install packages and import

In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

if code_dir_name not in str(Path.cwd()).split('/')[-1]:
    for _ in range(5):

        parent_path = str(Path.cwd().parents[_]).split('/')[-1]

        if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

            code_dir = str(Path.cwd().parents[_])

            if code_dir is not None:
                break
else:
    code_dir = str(Path.cwd())
sys.path.append(code_dir)

# %load_ext autoreload
# %autoreload 2


### Define some functions

In [ ]:
# Function to check for pop-ups
def popup(driver):

    # Signup popup
    try:
        signup_popup = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_class_name("modal_content")
        )
        if (signup_popup.is_displayed()) and (signup_popup.is_enabled()):
            print("Signup popup detected.")
            close_signup_popup = driver.find_element_by_id("prefix__icon-close-1")
            close_signup_popup.click()
            print("Signup popup dismissed.")
            # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'SearchForm')))
    except errors:

        # Feedback popup
        try:
            feedback_popup = WebDriverWait(driver, 10).until(
                lambda driver: driver.find_element_by_class_name(
                    "modal_main gdGrid common__modalContent___3TYQJ"
                )
            )
            if (feedback_popup.is_displayed()) and (feedback_popup.is_enabled()):
                # print('Feedback popup detected.')
                close_feedback_popup = driver.find_element_by_class_name(
                    "SVGInline modal_closeIcon"
                )
                close_feedback_popup.click()
                # print('Feedback popup dismissed.')
                # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'SearchForm')))
        except errors:

            # Accept Alert
            try:
                click_alert = driver.switch_to.alert()
                # print('Alert found.')
                click_alert.accept()
                # print('Alert accepted.')
                # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'SearchForm')))
            except errors:

                # Cookie popup
                try:
                    cookie_popup = WebDriverWait(driver, 10).until(
                        lambda driver: driver.find_element_by_id(
                            "onetrust-button-group"
                        )
                    )
                    if (cookie_popup.is_displayed()) and (cookie_popup.is_enabled()):
                        # print('Cookie detected.')
                        close_cookie_popup = driver.find_element_by_id(
                            "onetrust-accept-btn-handler"
                        )
                        if str(close_cookie_popup.text) == str("Accepteer cookies"):
                            close_cookie_popup.click()
                            # print('Cookie accepted.')
                        # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'SearchForm')))
                except errors:
                    pass
                except:
                    try:
                        print(
                            f"Cookie popup caused unexpected error: {sys.exc_info()[0]}"
                        )
                    except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
                        pass
            except:
                try:
                    print(f"Alert caused unexpected error: {sys.exc_info()[0]}")
                except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
                    pass
        except:
            try:
                print(f"Feedback popup caused unexpected error: {sys.exc_info()[0]}")
            except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
                pass
    except:
        try:
            print(f"Signup popup caused unexpected error: {sys.exc_info()[0]}")
        except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
            pass

    return


In [ ]:
# Find selected country and make sure country is Netherlands
def find_country(driver, soup, country):

    # Get html of page with beautifulsoup4
    # html = driver.page_source
    # print('Feeding html driver to BeautifulSoup.')
    # soup = BeautifulSoup(html, 'lxml')

    try:
        country_container = (
            soup.find("article", id="MainCol")
            .find("div", class_="css-1efvhw7 ee8wdg0")
            .find("div")
            .find("div", class_="pt pb-sm css-1ohf0ui")
            .find("div", class_="css-qo68e2")
        )
    except (NoSuchElementException, TimeoutException):
        print(f"No country selector found.")
    except errors:
        print(f"Unexpected error choosing {str(country)}: {sys.exc_info()[0]}.")
    else:
        country_chooser_soup = xpath_soup(
            country_container.find("div", class_="dropDownOptionsContainer").find("ul")
        )
        country_chooser = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_xpath(country_chooser_soup)
        )

        # Get html of page with beautifulsoup4
        # html = driver.page_source
        # print('Feeding html driver to BeautifulSoup.')
        # soup = BeautifulSoup(html, 'lxml')

        # country_selected = soup.find('option', selected = re.compile(r'.*')).text
        dropdown_container = country_container.find("div", class_="selectedLabel")
        country_selected = dropdown_container.text

        if str(country) != (country_selected):
            print(
                f"{str(country)} is not selected. {str(country_selected)} is selected."
            )
            dropdown_arrow_soup = xpath_soup(
                dropdown_container.find("span", class_="SVGInline arrowDown")
            )
            dropdown_arrow = WebDriverWait(driver, 10).until(
                lambda driver: driver.find_element_by_xpath(dropdown_arrow_soup)
            )
            dropdown_arrow.click()
            time.sleep(5)

            if dropdown_container.parent["aria-expanded"] == "true":

                # Get html of page with beautifulsoup4
                # html = driver.page_source
                # print('Feeding html driver to BeautifulSoup.')
                # soup = BeautifulSoup(html, 'lxml')

                countries_select_soup = [
                    countries_soup.find("span", class_="dropdownOptionLabel")
                    for countries_soup in country_chooser_soup.find_all("li")
                ]
                if any(str(x.text) == str(country) for x in countries_select_soup):
                    if str(x.text for x in countries_select_soup) == str(country):
                        country_select_soup = x

                elif all(str(x.text) != str(country) for x in countries_select_soup):
                    print(f"No match found for country {str(country)}.")
                    country = str(
                        input(
                            f"Please type one of the following country options in the input box:\n{str(countries_select_soup)}."
                        )
                    )
                    soup = find_country(driver, soup, country)

                country_select_xpath = xpath_soup(country_select_soup)
                country_select = WebDriverWait(driver, 10).until(
                    lambda driver: driver.find_element_by_xpath(country_select_xpath)
                )
                country_select.click()
                print(f"Selecting {str(country)}.")

                # Get html of page with beautifulsoup4
                # html = driver.page_source
                # print('Feeding html driver to BeautifulSoup.')
                # soup = BeautifulSoup(html, 'lxml')

            elif dropdown_container.parent["aria-expanded"] == "false":
                soup = find_country(driver, soup, country)

            else:
                print(f"Unexpected error choosing {str(country)}: {sys.exc_info()[0]}.")

        elif str(country) == str(country_selected):
            pass
        print(f"Searching for job ads in {str(country)}.")
        # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'SearchForm')))

    return soup


In [ ]:
# Function to find page number
def find_page_number(driver, soup, page_counter, keyword):
    # Get html of page with beautifulsoup4
    # html = driver.page_source
    # print('Feeding html driver to BeautifulSoup.')
    # soup = BeautifulSoup(html, 'lxml')

    try:
        page_footer = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located(
                (By.CSS_SELECTOR, "#MainCol > div.tbl.fill.px.my.d-flex")
            )
        )
        page_number_text = soup.find("div", class_="cell middle hideHH py-sm").text
        page_number_length = [int(s) for s in page_number_text.split() if s.isdigit()][
            1
        ]
        if int(page_number_length) == 1:
            print(f"Only 1 page found for {str(keyword)} jobs.")
            page_number = int(page_number_length)
            print(f"Current page number: {page_number}.")
        elif int(page_number_length) != 1:
            print(f"Total of {page_number_length} pages found for {str(keyword)} jobs.")
            page_number = [int(s) for s in page_number_text.split() if s.isdigit()][0]
            print(f"Current page number: {page_number}.")
    except errors:
        page_number = page_counter
        page_number_length = 99
        print(
            f"No page number indicator found. Page number set to counter: {page_number}"
        )

    return page_number_length, page_number


In [ ]:
# Function to click tabs
def click_tab(driver, soup, tab_name_key, tab_name_value):

    time.sleep(3)
    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    try:
        tab_exists = False
        pan_not_error = WebDriverWait(driver, 10).until_not(
            EC.visibility_of_element_located(
                (By.XPATH, '//div[contains(@class, "noPad opened transformNone")]')
            )
        )
        tabs_not_error = WebDriverWait(driver, 10).until_not(
            EC.visibility_of_element_located(
                (By.XPATH, '//*[@id="JDCol"]/div/div[2]/h3')
            )
        )
        if (pan_not_error) and (tabs_not_error):
            # Get current active tab
            WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.ID, "SerpFixedHeader"))
            )
            tab_scroll = soup.find("div", attrs={"data-test": "scrollable-tabs"})
            print(f"Target tab: {str(tab_name_key)} ({str(tab_name_value)}).")
            try:
                tab_soup = tab_scroll.find(
                    "div", attrs={"data-test": str(tab_name_key).strip()}
                )
            except:
                print(f"{str(tab_name_key)} ({tab_name_value}) tab does not exist.")
                tab_exists = False
            else:
                tab_exists = True

                if tab_soup != None:
                    xpath = xpath_soup(tab_soup)
                    try:
                        tab = WebDriverWait(driver, 10).until(
                            lambda driver: driver.find_element_by_xpath(xpath)
                        )
                        tab.click()
                    except:
                        print(
                            f"Could not click {str(tab_name_key)} ({str(tab_name_value)}) tab."
                        )
                        tab = WebDriverWait(driver, 10).until(
                            lambda driver: driver.find_element_by_xpath(xpath)
                        )
                        time.sleep(2)
                        tab.click()
                    else:
                        print(
                            f"Clicked {str(tab_name_key)} ({str(tab_name_value)}) tab."
                        )
                elif tab_soup is None:
                    print(
                        f"No {str(tab_name_key)} ({str(tab_name_value)}) tab on page."
                    )
                    tab_exists = False
        elif not tabs_not_error:
            if tabs_not_error.text == "Fout bij het laden":
                print(f"No tabs found.")
            else:
                if pan_not_error:
                    print(
                        f"Waiting for {str(tab_name_key)} ({tab_name_value}) tab. Resuming in 10 seconds: {sys.exc_info()[0]}."
                    )
                    soup, tab_exists = click_tab(
                        driver, soup, tab_name_key, tab_name_value
                    )
        elif not pan_not_error:
            print(f"No pan found.")
    except errors:
        print(
            f"Unexpected error going to {str(tab_name_key)} ({tab_name_value}) tab: {sys.exc_info()[0]}."
        )

    return soup, tab_exists


In [ ]:
# Function to get job data from company tab
def company_tab(driver, soup, info_key, info_value):

    pattern = re.compile(r"[^A-Za-z0-9, ]+")

    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    # WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#CompanyContainer')))
    try:
        print("-" * 20)
        info_row = soup.find("div", id="EmpBasicInfo").find("div").find("div")
    except NoSuchElementException:
        print(f"No {info_key} ({info_value}) found.")
        info_row = -1
    except errors:
        print(
            f"Unexpected error getting {info_key} ({info_value}): {sys.exc_info()[0]}"
        )
        info_text = -1
    else:
        for info in info_row:
            try:
                info_label = info.find("span").text
            except:
                pass
            else:
                if str(info_label) == str(info_value):
                    info_text = str(info.find_all("span")[1].text).strip()
                    print(f"{info_key}: {info_text}")
                    print("-" * 20)
                else:
                    pass
    try:
        return info_text
    except:
        return -1


In [ ]:
# Collect individual job links
def get_each(
    driver,
    main_window,
    window_before,
    keyword,
    keyword_file,
    site,
    jobs,
    job_soup,
    job_id,
    jobs_count,
    page_counter,
    page_number,
    page_number_length,
    save_path,
    json_file_name,
):
    jobs = remove_dupe_dicts(jobs)

    # If results open in new window, make sure everything loads
    new_window = check_window(driver, main_window, window_before)

    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    # Get job id, url and age
    print("-" * 20)
    print(f"Job ID: {job_id}")
    print("-" * 20)
    job_url = "https://www.glassdoor.nl" + str(
        job_soup.find("div", class_="d-flex flex-column css-x75kgh e1rrn5ka3").find(
            "a"
        )["href"]
    )
    print(f"Job URL: {job_url}")
    print("-" * 20)
    job_age = job_soup.find(
        "div",
        class_="d-flex align-items-end pl-std css-mi55ob",
        attrs={"data-test": "job-age"},
    ).text
    for s in job_age.split():
        job_age_number = int(s) if s.isdigit() else job_age
    for i in re.findall("[\w +/.]", job_age):
        job_age_unit = str(i) if i.isalpha() else job_age
    print(f"Job Age: {job_age}")
    print("-" * 20)

    #########################################################
    #########################################################
    ####################### TAB INFO #######################
    #########################################################
    #########################################################

    tab_name = {"job": "Vacature", "overview": "Bedrijf", "rating": "Beoordeling"}

    #########################################################
    ##################### JOB OFFER TAB #####################
    #########################################################

    # Click job offer tab
    try:
        soup, job_offer_tab_exists = click_tab(
            driver, soup, list(tab_name.keys())[0], list(tab_name.values())[0]
        )
    except NoSuchElementException:
        print(
            f"No {list(tab_name.keys())[0]} tab - {sys.exc_info()[0]}.\nNo job title, job description, company name, company location, company url."
        )
        job_title, job_description, company_name, location = assign_all(4, -1)
    else:
        if job_offer_tab_exists:
            # Get main data
            try:
                (
                    job_title,
                    job_description,
                    company_name,
                    location,
                    company_url,
                ) = get_main(driver, soup, job_soup)
            # In case there is no company tab, -1 for all variables
            except NoSuchElementException:
                print(
                    f"No {list(tab_name.keys())[0]} tab - {sys.exc_info()[0]}.\nNo job title, job description, company name, company location, company url."
                )
                (
                    job_title,
                    job_description,
                    company_name,
                    location,
                    company_url,
                ) = assign_all(5, -1)
        elif not job_offer_tab_exists:
            print(
                f"{list(tab_name.keys())[0]} tab doesn't exist. Refereshing and trying again."
            )
            driver.refresh()
            time.sleep(10)
            try:
                # Click job offer tab
                soup, job_offer_tab_exists = click_tab(
                    driver, soup, list(tab_name.keys())[0], list(tab_name.values())[0]
                )
                if job_offer_tab_exists:
                    (
                        job_title,
                        job_description,
                        company_name,
                        location,
                        company_url,
                    ) = get_main(driver, soup, job_soup)
                elif not job_offer_tab_exists:
                    (
                        job_title,
                        job_description,
                        company_name,
                        location,
                        company_url,
                    ) = assign_all(5, -1)
            except:
                (
                    job_title,
                    job_description,
                    company_name,
                    location,
                    company_url,
                ) = assign_all(5, -1)

    #########################################################
    ##################### COMPANY TAB #####################
    #########################################################

    try:
        # Click company tab
        soup, company_tab_exists = click_tab(
            driver, soup, list(tab_name.keys())[1], list(tab_name.values())[1]
        )
    except NoSuchElementException:
        print(
            f"No {list(tab_name.keys())[1]} tab - {sys.exc_info()[0]}.\nNo industry, sector, company size, company type."
        )
        industry, sector, company_size, company_type = assign_all(4, -1)
    else:
        if company_tab_exists:
            # Get info in tabs
            company_tab_info = {
                "Industry": "Bedrijfstak",
                "Sector": "Sector",
                "Size": "Grootte",
                "Type": "Type",
            }

            # Getting industry
            industry = company_tab(
                driver,
                soup,
                list(company_tab_info.keys())[0],
                list(company_tab_info.values())[0],
            )

            # Getting sector
            sector = company_tab(
                driver,
                soup,
                list(company_tab_info.keys())[1],
                list(company_tab_info.values())[1],
            )

            # Getting company size
            company_size = company_tab(
                driver,
                soup,
                list(company_tab_info.keys())[2],
                list(company_tab_info.values())[2],
            )

            # Getting company type
            company_type = company_tab(
                driver,
                soup,
                list(company_tab_info.keys())[3],
                list(company_tab_info.values())[3],
            )

        elif not company_tab_exists:
            print(
                f"{list(tab_name.keys())[1]} tab doesn't exist. Moving on to next tab."
            )
            industry, sector, company_size, company_type = assign_all(4, -1)

    #########################################################
    ###################### RATINGS TAB #####################
    #########################################################

    try:
        # Click ratings tab
        soup, rating_tab_exists = click_tab(
            driver, soup, list(tab_name.keys())[2], list(tab_name.values())[2]
        )
    except NoSuchElementException:
        print(f"No {list(tab_name.keys())[2]} tab - {sys.exc_info()[0]}.")
        rating = -1
    else:
        if rating_tab_exists:
            try:
                print("-" * 20)
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "#RatingContainer"))
                )
                try:
                    rating = (
                        driver.find_element_by_xpath(
                            '//*[@id="RatingContainer"]/div[1]/div/div[1]/div[1]'
                        ).text.split()
                    )[0]
                except:
                    rating = (job_soup.find("div").find("span").text.split())[0]
                print(f"Rating average: {rating}")
                print("-" * 20)
            except NoSuchElementException:
                print(f"No job rating found on {list(tab_name.keys())[2]} tab.")
                rating = -1
            except errors:
                print(
                    f"Unexpected error getting {list(tab_name.keys())[2]}: {sys.exc_info()[0]}."
                )
                rating = -1

        elif not rating_tab_exists:
            print(
                f"{list(tab_name.keys())[1]} tab doesn't exist. Moving on to next tab."
            )
            rating = -1

    # Save to dict
    print("Saving to dict...")
    today = datetime.date.today()
    jobs.append(
        {
            "Search Keyword": str(keyword),
            "Platform": site,
            "Job ID": int(job_id),
            "Job Title": str(job_title),
            "Company Name": str(company_name),
            "Location": str(location),
            "Industry": str(industry),
            "Sector": str(sector),
            "Job Description": str(job_description),
            "Type of ownership": str(company_type),
            "Rating": float(rating),
            "Company URL": company_url,
            "Job URL": job_url,
            "Job Age": job_age,
            "Job Age Number": job_age_number,
            "Collection Date": (today.strftime("%Y-%m-%d")),
        }
    )

    # Save dict as json file
    jobs = remove_dupe_dicts(jobs)
    try:
        print(
            f"Progress for {keyword} job ads (page {page_number}): {len(jobs)} / {int(jobs_count)*int(page_number_length)}"
        )
    except:
        print(f"Progress for {keyword} job ads (page {page_number}): {len(jobs)}")
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    # Save page counter
    if page_counter > 0:
        with open(
            save_path + f"{site.lower()}_page_counter_{keyword_file}.txt", "w"
        ) as f:
            f.write(str(page_counter))

    # Check current window and go back if required
    check_window_back(driver, main_window, window_before, new_window)

    return jobs


In [ ]:
# Function to get main job data
def get_main(driver, soup, job_soup):

    pattern = re.compile(r"[^A-Za-z0-9, ]+")
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "JDCol")))

    try:
        job_title = driver.find_element_by_xpath(
            '//*[@id="MainCol"]/div[1]/ul/li[1]/div[2]/a/span'
        ).text.strip()
    except NoSuchElementException:
        print(f"No job title found.")
        job_title = -1
    except errors:
        print(f"Unexpected error getting job title: {sys.exc_info()[0]}.")
        job_title = -1
    else:
        print("-" * 20)
        print(f"Job Title: {str(job_title)}")
        print("-" * 20)

    try:
        job_description_list = []
        job_desc = (
            soup.find("div", id="JobDescriptionContainer")
            .find("div")
            .find("div", class_="jobDescriptionContent desc")
            .find_all("div")
        )
        if len(job_desc) == 0:
            job_desc_pattern = re.compile(r"JobDesc\d+")
            job_desc = driver.find_element_by_xpath(
                '//*[@id="JobDescriptionContainer"]'
            ).find_elements_by_xpath(f'//*[@id="{job_desc_pattern}"]/div')
        for job_d in job_desc:
            job_description_list.append(job_d.text.strip())
        job_description = "\n".join(job_description_list)
    except NoSuchElementException:
        print(f"No job description found.")
        job_description = -1
    except errors:
        print(f"Unexpected error getting job description: {sys.exc_info()[0]}.")
        job_description = -1
    else:
        print("-" * 20)
        print(f"Job Description: {str(job_description[:50])}")
        print("-" * 20)

    try:
        company_name = (
            job_soup.find(
                "div", class_="d-flex flex-column pl-sm css-1d3xmk8 e1rrn5ka4"
            )
            .find("div", class_="d-flex justify-content-between align-items-start")
            .find("a", class_="css-l2wjgv e1n63ojh0 jobLink")
            .find("span")
            .text.strip()
        )
    except NoSuchElementException:
        print(f"No job company name found.")
        company_name = -1
    except errors:
        print(f"Unexpected error getting company name: {sys.exc_info()[0]}.")
        company_name = -1
    else:
        print("-" * 20)
        print(f"Company Name: {company_name}")
        print("-" * 20)

    try:
        location = (
            job_soup.find(
                "div", class_="d-flex flex-column pl-sm css-1d3xmk8 e1rrn5ka4"
            )
            .find("div", class_="d-flex flex-wrap css-11d3uq0 e1rrn5ka2")
            .find("span")
            .text.strip()
        )
    except NoSuchElementException:
        print(f"No job company location found.")
        location = -1
    except errors:
        print(f"Unexpected error getting company location: {sys.exc_info()[0]}.")
        location = -1
    else:
        print("-" * 20)
        print(f"Company Location: {str(location)}")
        print("-" * 20)

    try:
        company_url = "https://www.glassdoor.nl/" + str(
            driver.find_element_by_xpath(
                '//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[2]/div/div[1]/div[1]/a'
            ).get_attribute("href")
        )
    except NoSuchElementException:
        print(f"No job company url found.")
        company_url = -1
    except errors:
        print(f"Unexpected error getting company url: {sys.exc_info()[0]}.")
        company_url = -1
    else:
        print("-" * 20)
        print(f"Company URL: {company_url}")
        print("-" * 20)

    # Check if collection successful
    imp_collected = [job_title, job_description, company_name, location, company_url]
    if all((var != int(-1)) or (var != str("-1")) for var in imp_collected):
        print("Main data collected successfully.")

    elif any((var == int(-1)) or (var == str("-1")) for var in imp_collected):
        print(
            f"Main data NOT collected successfully. Sleeping for 10 seconds then trying again."
        )
        time.sleep(10)
        try:
            job_title, job_description, company_name, location, company_url = get_main(
                driver, soup
            )
        except:
            print(
                f"Main data NOT collected successfully. Not attempting again. Collecting remaining data: {sys.exc_info()[0]}."
            )

    return job_title, job_description, company_name, location, company_url


In [ ]:
def get_page_jobs(
    driver,
    soup,
    keyword,
    keyword_file,
    site,
    main_window,
    window_before,
    page_counter,
    page_number,
    page_number_length,
    jobs,
    df_old_jobs,
    save_path,
    json_file_name,
):

    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    # try:
    # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'SearchForm')))
    try:
        jobs_on_page = (
            soup.find("div", id="JobResults")
            .find("article", id="MainCol")
            .find("ul", class_="hover p-0 css-7ry9k1 exy0tjh5")
            .find_all("li")
        )
        jobs_count = len(jobs_on_page)
    except AttributeError as e:
        print(
            f"Unexpected error with job links: {e} - {sys.exc_info()[0]}.\nAttempting .children method."
        )
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (
                    By.CSS_SELECTOR,
                    "#MainCol > div:nth-child(1) > ul > li.react-job-listing.css-7x0jr.eigr9kq3",
                )
            )
        )
        jobs_grid = soup.find_all("ul", class_="hover p-0  css-7ry9k1 exy0tjh5")
        jobs_on_page = [job.children for job in jobs_grid]
        jobs_count = cardinality.count(jobs_on_page)

    if int(jobs_count) != 0:
        print(
            f"{int(jobs_count)} job(s) found on page {page_number} out of {page_number_length} page(s)."
        )
        try:
            print(
                f"Progress for {keyword} job ads (page {page_number}): {len(jobs)} / {int(jobs_count)*int(page_number_length)}"
            )
        except:
            print(f"Progress for {keyword} job ads (page {page_number}): {len(jobs)}")

        for job_soup in jobs_on_page:
            print("-" * 20)
            print(
                f"JOB NUMBER: {int(jobs_on_page.index(job_soup)) + 1} OUT OF {int(jobs_count)} ON PAGE {page_number} OUT OF {page_number_length}."
            )
            page_counter = page_number
            if page_counter > 0:
                with open(
                    save_path + f"{site.lower()}_page_counter_{keyword_file}.txt", "w"
                ) as f:
                    f.write(str(page_counter))
            print("-" * 20)
            job_xpath = xpath_soup(job_soup)
            job_button = WebDriverWait(driver, 10).until(
                lambda driver: driver.find_element_by_xpath(job_xpath)
            )

            try:
                job_button.click()
            except errors:
                print(
                    f"Click did not work. Trying again to get job soup for {str(keyword)}."
                )
                # Get html of page with beautifulsoup4
                # html = driver.page_source
                # print('Feeding html driver to BeautifulSoup.')
                # soup = BeautifulSoup(html, 'lxml')
                job_button.click()

            # Get job id
            job_id = int(job_soup["data-id"])

            # Check if already collected
            jobs, job_present = id_check(
                driver,
                main_window,
                window_before,
                keyword,
                site,
                df_old_jobs,
                jobs,
                job_soup,
                job_id,
                jobs_count,
                save_path,
                json_file_name,
            )
            if job_present is False:
                jobs = get_each(
                    driver,
                    main_window,
                    window_before,
                    keyword,
                    keyword_file,
                    site,
                    jobs,
                    job_soup,
                    job_id,
                    jobs_count,
                    page_counter,
                    page_number,
                    page_number_length,
                    save_path,
                    json_file_name,
                )
            elif job_present is True:
                pass

    elif int(jobs_count) == 0:
        print(
            f"No jobs results found for {str(keyword)}. No error detected. Moving on to next search."
        )

    # except:
    #     print(f'No jobs results found for {str(keyword)}: {sys.exc_info()[0]}. Moving on to next search.')

    # Save dict as json file
    jobs = remove_dupe_dicts(jobs)
    print("-" * 20)
    try:
        print(
            f"Progress for {keyword} job ads (page {page_number}): {len(jobs)} / {int(jobs_count)*int(page_number_length)}"
        )
    except:
        print(f"Progress for {keyword} job ads (page {page_number}): {len(jobs)}")
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    print("-" * 20)
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    # End of page
    print("-" * 20)
    print(f"REACHED END OF JOBS FOR PAGE: {page_number}.")
    print("-" * 20)

    # Save page counter
    page_counter += 1
    if page_counter > 0:
        with open(
            save_path + f"{site.lower()}_page_counter_{keyword_file}.txt", "w"
        ) as f:
            f.write(str(page_counter))

    # Save dict as json file
    jobs = remove_dupe_dicts(jobs)
    try:
        print(
            f"Progress for {keyword} job ads (page {page_number}): {len(jobs)} / {int(jobs_count)*int(page_number_length)}"
        )
    except:
        print(f"Progress for {keyword} job ads (page {page_number}): {len(jobs)}")
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    # Save the old page counter
    if page_counter > 0:
        with open(
            save_path + f"{site.lower()}_page_counter_{keyword_file}.txt", "w"
        ) as f:
            f.write(str(page_counter))

    print(f"End of data collection for {keyword}.\nGot {len(jobs)} jobs.")

    # Save dict as json file
    jobs = remove_dupe_dicts(jobs)
    print("-" * 20)
    try:
        print(
            f"Progress for {keyword} job ads (page {page_number}): {len(jobs)} / {int(jobs_count)*int(page_number_length)}"
        )
    except:
        jobs_count = len(jobs)
        print(
            f"Progress for {keyword} job ads (page {page_number}): {len(jobs)} / {int(jobs_count)*int(page_number_length)}"
        )
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    print("-" * 20)
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    return jobs, jobs_count, page_counter, page_number


### Get-jobs function

In [ ]:
# MAIN FUNCTION TO GET JOB ADS
def get_jobs(
    driver,
    keyword,
    keyword_url,
    keyword_file,
    site,
    country,
    page_counter,
    main_window,
    df_old_jobs,
    jobs,
    save_path,
    json_file_name,
    df_file_name,
):

    url = f"https://www.glassdoor.nl/sitedirectory/title-jobs.htm"
    jobs_count = len(jobs)

    # Go to page and establish main windows
    print(f"Going to url: {url}")
    driver.get(url)
    window_before = driver.window_handles[0]
    print(f"First Window: {window_before}")
    print(f"Current page title: {driver.title}")

    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    # Input keyword in search bar
    try:
        print(f"Searching for {keyword.upper()} job ads.")
        # Get html of page with beautifulsoup4
        # html = driver.page_source
        # print('Feeding html driver to BeautifulSoup.')
        # soup = BeautifulSoup(html, 'lxml')

        search_bar = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_id("sc.keyword")
        )
        search_bar.send_keys([keyword])
        search_bar.submit()
        search_button = WebDriverWait(
            driver, 10, ignored_exceptions=StaleElementReferenceException
        ).until(EC.presence_of_element_located((By.ID, "HeroSearchButton")))
        driver.execute_script("arguments[0].click();", search_button)
    except errors:
        print(
            f"Unexpected error with search: {sys.exc_info()[0]}.\nAttempting search again..."
        )

        # Get html of page with beautifulsoup4
        # html = driver.page_source
        # print('Feeding html driver to BeautifulSoup.')
        # soup = BeautifulSoup(html, 'lxml')

        print("Clicking search button manually.")
        try:
            search_bar.send_keys(Keys.ENTER)
        except TimeoutException:
            search_button = WebDriverWait(
                driver, 10, ignored_exceptions=StaleElementReferenceException
            ).until(EC.presence_of_element_located((By.ID, "HeroSearchButton")))
            driver.execute_script("arguments[0].click();", search_button)
        except StaleElementReferenceException:
            pyautogui.click(1170, 230)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "PageContent"))
            )
        except errors:
            print(
                f"Unexpected error with clicking the search button: {sys.exc_info()[0]}."
            )
            pyautogui.click(1170, 190)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "PageContent"))
            )
        except:
            pass

    if page_counter > 1:
        url = (f"_IP{page_counter}" + str(".htm")).join(
            str(driver.current_url).split(str(".htm"))
        )
        # Go to page and establish main windows
        print(f"Going to url: {url}")
        driver.get(url)
        window_before = driver.window_handles[0]
        print(f"First Window: {window_before}")
        print(f"Current page title: {driver.title}")

        # Get html of page with beautifulsoup4
        # html = driver.page_source
        # print('Feeding html driver to BeautifulSoup.')
        # soup = BeautifulSoup(html, 'lxml')
    elif page_counter <= 1:
        pass

    # Find selected country and make sure country is Netherlands
    soup = find_country(driver, soup, country)

    # Find page number
    page_number_length, page_number = find_page_number(
        driver, soup, page_counter, keyword
    )

    # Check for errors
    try:
        not_page_error = WebDriverWait(driver, 10).until_not(
            lambda driver: driver.find_element_by_class_name("noResults")
        )
    except TimeoutException:
        print(f"No jobs results found for {str(keyword)}. Moving on to next search.")
    else:
        while not_page_error:
            # Find job button next siblings with bs4
            print("Finding job links in page.")
            jobs, jobs_count, page_counter, page_number = get_page_jobs(
                driver,
                soup,
                keyword,
                keyword_file,
                site,
                main_window,
                window_before,
                page_counter,
                page_number,
                page_number_length,
                jobs,
                df_old_jobs,
                save_path,
                json_file_name,
            )

    #             # Click on next page button
    #             try:
    #                 if int(page_number_length) == int(page_number):
    #                     print(f'Last page reached at page {page_number}. End of data collection for {keyword}.\nGot {len(jobs)} jobs.')
    #                     break
    #                 elif int(page_number_length) != int(page_number):
    #                     try:
    #                         driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    #                         next_page_button = WebDriverWait(driver, 10).until_not(lambda driver: driver.find_element_by_xpath('//*[@id="FooterPageNav"]/div/ul/li[7]/a'))
    #                     except (AttributeError, TimeoutException):
    #                         driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    #                         # Get html of page with beautifulsoup4
    #                         html = driver.page_source
    #                         print('Feeding html driver to BeautifulSoup.')
    #                         soup = BeautifulSoup(html, 'lxml')
    #                         next_page_button = WebDriverWait(driver, 10).until_not(lambda driver: driver.find_element_by_xpath(xpath_soup(soup.find('a', attrs={'data-test': 'pagination-next'}))))
    #                     except:
    #                         next_page_button = WebDriverWait(driver, 10).until_not(lambda driver: driver.find_element_by_xpath(f'//*[@id="FooterPageNav"]/div/ul/li[{str(int(page_number) + 2)}]/a'))
    #                     finally:
    #                         if not isinstance(next_page_button, bool):
    #                             pass
    #                         elif isinstance(next_page_button, bool):
    #                             print('Next button is boolean. Clicking using page number.')
    #                             next_page_button = WebDriverWait(driver, 10).until_not(lambda driver: driver.find_element_by_xpath(f'//*[@id="FooterPageNav"]/div/ul/li[{str(int(page_number) + 1)}]/a'))
    #                         next_page_button.click()
    #                         print(f'Going to next page {int(page_number)+1}.\nGot {len(jobs)} jobs.')
    #                     # Find page number
    #                     page_number_length, page_number = find_page_number(driver, soup, page_counter, keyword)

    #                     # Get jobs
    #                     jobs, jobs_count, page_counter, page_number = get_page_jobs(driver, soup, keyword, keyword_file, site, main_window, window_before, page_counter, page_number, page_number_length, jobs, df_old_jobs, save_path, json_file_name)
    #             except (NoSuchWindowException, NoSuchElementException):
    #                 print(f'Could not go to next page. End of data collection for {keyword}.\nGot {len(jobs)} jobs.')

    ############################################### FINAL SAVING AND CLEANUP ###############################################

    # Save dict as json file
    print("Final save.")
    jobs = remove_dupe_dicts(jobs)
    try:
        print(
            f"Progress for {keyword} job ads (page {page_number}): {len(jobs)} / {int(jobs_count)*int(page_number_length)}"
        )
    except:
        print(f"Progress for {keyword} job ads (page {page_number}): {len(jobs)}")
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    # Save jobs to df
    print(f"Saving {keyword} jobs data to df...")
    if jobs:
        df_jobs = pd.DataFrame(jobs)
    elif not jobs:
        print(f"Jobs dict for {keyword} is empty.")
        df_jobs = pd.DataFrame()

    # Save df as csv
    if not df_jobs.empty:
        df_jobs.append(df_old_jobs, ignore_index=True)
        df_jobs = clean_df(df_jobs)
        df_jobs = save_df(
            keyword, df_jobs, save_path, keyword_file.lower(), df_file_name.lower()
        )
    elif df_jobs.empty:
        df_jobs = df_old_jobs

    return jobs, jobs_count, df_jobs


### Run get_jobs function

In [ ]:
# if __name__ == '__main__':
def run_all_glassdoor(
    site="Glassdoor",
    language="English",
    country="Nederland",
    incognito_enabled=True,
    headless_enabled=False,
    proxy_enabled=False,
    logging_enabled=False,
    print_enabled=True,
):

    # Get keywords
    (
        keywords_list,
        keywords_sector,
        keywords_womenvocc,
        keywords_menvocc,
        keywords_genvsect,
        keywords_oldvocc,
        keywords_youngvocc,
        keywords_agevsect,
    ) = get_keyword_list(print_enabled=print_enabled)
    # remove_list = ['primary school teacher', 'selling real estate', 'physiotherapist']
    # for k in remove_list:
    #     keywords_list.remove(k)

    # Start driver
    driver = get_driver(select_driver)

    # Start for-loop
    #     keywords_oldvocc = ['industry']
    keywords_list = keywords_oldvocc + keywords_youngvocc
    for keyword in keywords_youngvocc:
        recollect = False

        # Check internet connection
        if not is_connected(driver):
            is_connected(driver)

        # Get main info
        (
            keyword_url,
            keyword_file,
            save_path,
            json_file_name,
            df_file_name,
            logs_file_name,
            filemode,
        ) = main_info(keyword, site)

        # Set up log
        if logging_enabled is True:
            print("Logging enabled.")
            writer = MyWriter(logs_file_name.lower(), filemode)
            sys.stdout = writer
            logger = logging.getLogger(__name__)
            logging.basicConfig(
                filename=writer.LOGS_PATH + logs_file_name.lower(),
                filemode=filemode,
                level=logging.CRITICAL,
                format="%(asctime)s:%(levelname)s: %(message)s",
            )
        elif logging_enabled is False:
            print("No logging enabled.")

        # Load and merge existing dict and df
        print(f"All files will be saved in folder: {save_path}{json_file_name}")
        jobs, df_old_jobs = load_merge_dict_df(
            keyword, save_path, df_file_name, json_file_name
        )

        # Beging data collection
        print(
            "-" * 20,
            f"BEGINING OF DATA COLLECTION FOR {keyword.upper()} JOB ADS",
            "-" * 20,
        )
        main_window = driver.current_window_handle
        print(f"Main Window: {main_window}")

        # Load and save page_counter
        page_counter_path = (
            save_path + f"{site.lower()}_page_counter_{keyword_file}.txt"
            if recollect is False
            else save_path + f"{site.lower()}_page_counter_done_{keyword_file}.txt"
        )

        if is_non_zero_file(page_counter_path) is True:
            with open(page_counter_path) as f:
                page_counter = int(f.read())
        elif is_non_zero_file(page_counter_path) is False:
            page_counter = 1
        with open(page_counter_path, "w") as f:
            f.write(str(page_counter))

        print(f"Starting collection from page number: {page_counter}")

        # Start threads
        popup_thread(driver, popup, popup_checker)
        # stale_element_thread(driver, stale_element, stale_element_checker)
        # act_cool_thread(driver, act_cool, act_cool_checker)

        # Get jobs df
        jobs, jobs_count, df_jobs = get_jobs(
            driver,
            keyword.lower(),
            keyword_url,
            keyword_file,
            site,
            country,
            page_counter,
            main_window,
            df_old_jobs,
            jobs,
            save_path.lower(),
            json_file_name.lower(),
            df_file_name.lower(),
        )

        # Save df as csv
        df_jobs = save_df(
            keyword, df_jobs, save_path, keyword_file.lower(), df_file_name.lower()
        )
        print(
            "-" * 20, f"END OF DATA COLLECTION FOR {keyword.upper()} JOB ADS", "-" * 20
        )

    # Close and quit driver
    print(f"Closing driver.")
    driver.close()
    driver.quit()
    # thread_popup_checker.terminate()
    # thread_stale_element_checker.terminate()
    # thread_act_cool_checker.terminate()
    # sys.stdout.close()
    # sys.exit()

    return df_jobs


In [ ]:
df_jobs_glassdoor = run_all_glassdoor()


In [ ]:
# keyword = 'artsen'
# site = 'Glassdoor'
# keyword_from_list = True
# site_from_list = False
# df_jobs = post_cleanup(keyword, site)
